In [1]:
import requests
from lxml import etree
from typing import List
import arrow
import datetime
import pandas as pd
import numpy as np
import pathlib

In [21]:
df1=pd.DataFrame({'A':[1,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,2,3,3,np.nan,1,2]})
df2=pd.DataFrame({'A':[1,3,2,2,3,2,3,4,2,3,3,2,np.nan,]})

In [22]:
df1

,A
0,1.0
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,2.0
8,3.0
9,3.0


In [7]:
df1.iloc[1:4]['A']='-999'

C:\Users\evase\Anaconda3\envs\new_oil_gdal\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
for val in range(5,7):
    df1.iloc[val]['A']='-999'

In [18]:
df1

,A
0,1.0
1,NaN
2,NaN
3,NaN
4,NaN
5,-999.0
6,-999.0
7,2.0
8,3.0
9,3.0


In [20]:
def conduct_nan(df: pd.DataFrame, count_max: int, nan_val: str, col_name: str):
    '''
        处理 df 中的nan
        对于连续 len 的 nan，找到最后的nan index，并将 [index-len,index] 均设置为 nan_val，直到连续的 nan < len 为止
    '''
    df_copy = df.copy()
    if len(df_copy) < count_max:
        return
    for index in range(len(df_copy)):
        temp = df_copy.iloc[index][col_name]
        if pd.isna(temp) and len(df_copy) > index + 1:
            count = 0
            count = count + 1
            next_index = 0
            last_index = 0
            while True:
                next_index = next_index + 1
                # TODO:[*] 若最后一个是 nan
                # IndexError: single positional indexer is out-of-bounds
                next_temp = df_copy.iloc[index + next_index][col_name]
                if pd.isna(next_temp):
                    count = count + 1
                    # last_index = next_index
                    # if count > count_max:
                    #     #  注意此处应该是将所有的 在 [index,last_index] 均设置为 nan_val
                    #     df.iloc[index: next_index][col_name] = nan_val
                    # else:
                    #     pass
                else:
                    break
            if count > count_max:
                #  注意此处应该是将所有的 在 [index,last_index] 均设置为 nan_val
                for df_index in range(index, count + 1):
                    df_copy.iloc[df_index][col_name] = nan_val

    return df_copy

In [25]:
df_new=conduct_nan(df1,4,'99998','A')

In [ ]:
np.

In [30]:
df_new.iloc[2]==np.float64(99998)

A    True
Name: 2, dtype: bool

In [29]:
np.float64(99998)

99998.0

In [27]:
df_new

,A
0,1.0
1,99998.0
2,99998.0
3,99998.0
4,99998.0
5,99998.0
6,99998.0
7,2.0
8,3.0
9,3.0


In [31]:
df_new[df_new==np.float64(99998)]

,A
0,NaN
1,99998.0
2,99998.0
3,99998.0
4,99998.0
5,99998.0
6,99998.0
7,NaN
8,NaN
9,NaN


In [34]:
df_new[df_new!=np.float64(99998)]

,A
0,1.0
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,2.0
8,3.0
9,3.0


In [35]:
df_new.interpolate(method='linear',limit=60,limit_direction ='backward')

,A
0,1.0
1,99998.0
2,99998.0
3,99998.0
4,99998.0
5,99998.0
6,99998.0
7,2.0
8,3.0
9,3.0


In [37]:
df = pd.DataFrame({'CST_NO': [11, 22, 11, 22, 11, 22],
                'AMOUNT': [10, 20, 30, np.nan, 1000, 2000],
                'START_TIME':['2016-08-21 17:15:15','2016-08-21 22:22:22',
                             '2016-08-22 16:16:16','2016-08-22 20:23:23',
                              '2016-08-23 15:12:12','2016-08-23 13:12:12'],
                'CLIENT_IP':['192.1', '192.2', '192.1', '192.3', '192.2','192.3'],
                'CLIENT_MAC':['ios', 'win', 'win', 'ios', 'ios','win'],
                'TO':['WEI', 'BAI', 'WEI', 'ZHU', 'WANG','ZHU'],
                })

In [38]:
df

,CST_NO,AMOUNT,START_TIME,CLIENT_IP,CLIENT_MAC,TO
0,11,10.0,2016-08-21 17:15:15,192.1,ios,WEI
1,22,20.0,2016-08-21 22:22:22,192.2,win,BAI
2,11,30.0,2016-08-22 16:16:16,192.1,win,WEI
3,22,NaN,2016-08-22 20:23:23,192.3,ios,ZHU
4,11,1000.0,2016-08-23 15:12:12,192.2,ios,WANG
5,22,2000.0,2016-08-23 13:12:12,192.3,win,ZHU


In [40]:
df.loc[2,'AMOUNT']=999

In [41]:
df

,CST_NO,AMOUNT,START_TIME,CLIENT_IP,CLIENT_MAC,TO
0,11,10.0,2016-08-21 17:15:15,192.1,ios,WEI
1,22,20.0,2016-08-21 22:22:22,192.2,win,BAI
2,11,999.0,2016-08-22 16:16:16,192.1,win,WEI
3,22,NaN,2016-08-22 20:23:23,192.3,ios,ZHU
4,11,1000.0,2016-08-23 15:12:12,192.2,ios,WANG
5,22,2000.0,2016-08-23 13:12:12,192.3,win,ZHU
